## loading the XES file as dict with lxml

In [1]:
from lxml import objectify
import dateutil.parser
import pickle
import os


def load_as_xml():
    xes_as_xml = objectify.fromstring(open("BPI Challenge 2018.xes", 'r').read())
    return xes_as_xml


def load_as_dict(year):
    json_file = "BP_Challenge_2018-%s.pickle" % year

    # se o arquivo ja existe, apenas carregar e returnar o conteudo
    if os.path.isfile(json_file):
        print "Loading data from file"
        return pickle.load(open(json_file))

    # carregando arquivo .XES
    xes_xml = objectify.fromstring(open("BPI Challenge 2018.xes", 'r').read())
    print "XES file loaded"

    xes_as_dict = {}
    for trace in xes_xml.trace:
        # convertendo elementos
        trace_dict = extract_basic_elements(trace)

        if int(trace_dict['year']) != year:
            continue

        # convertendo e salvando em lista os eventos
        trace_dict['event'] = [extract_basic_elements(e) for e in trace.event]
        # ordenando eventos pela data
        trace_dict['event'].sort(key=lambda x: x['time:timestamp'])

        xes_as_dict[trace_dict['identity:id']] = trace_dict
    
    print "Dict file created"
    # liberando memoria
    del xes_xml

    # salvando dict em arquivo para evitar futuros reprocessamentos
    with open(json_file, 'w') as f:
      pickle.dump(xes_as_dict, f)

    print "Dict file saved as %s" % json_file
    return xes_as_dict


def extract_basic_elements(element):
    d_elem = {}

    if element.xpath('id'):
        for id_element in element.id:
            d_elem[id_element.get('key')] = id_element.get('value')

    if element.xpath('int'):
        for int_element in element.int:
            d_elem[int_element.get('key')] = int(int_element.get('value'))

    if element.xpath('string'):
        for str_element in element.string:
            d_elem[str_element.get('key')] = str_element.get('value')

    if element.xpath('boolean'):
        for bool_element in element.boolean:
            d_elem[bool_element.get('key')] = 1 if bool_element.get('value') == 'true' else 0

    if element.xpath('date'):
        for date_element in element.date:
            d_elem[date_element.get('key')] = dateutil.parser.parse(date_element.get('value'))

    if element.xpath('float'):
        for float_element in element.float:
            d_elem[float_element.get('key')] = float(float_element.get('value'))

    return d_elem


### Carregando os casos por ano

In [2]:
# APENAS 2017!!!
year = 2017  # basta alterar o ano e executar as celulas abaixo para gerar novos CSVs

In [3]:
xes_as_dict_year = load_as_dict(year)

XES file loaded
Dict file created
Dict file saved as BP_Challenge_2018-2017.pickle


In [4]:
qtd_trace = len(xes_as_dict_year.keys())
print "Total de casos: ", qtd_trace

Total de casos:  14507


In [5]:
# convertendo em lista de casos e filtrando
casos_year = []

for key, caso in xes_as_dict_year.items():
    # se o caso tem pelo menos um evento registrado
    if caso.get('event'):

        # se o ultimo evento foi o finish payment (1) ou nao (0)
        caso['finish payment'] = 1 if caso['event'][-1]['activity'] == 'finish payment' else 0
        # data de registro do primeiro evento
        caso['start_time'] = caso['event'][0]['time:timestamp']
        # data de registro do ultimo evento
        caso['end_time'] = caso['event'][-1]['time:timestamp']
        # total de eventos
        caso['tot_events'] = len(caso['event'])
        # intervalo de tempo entre o primeiro e ultimo evento
        delta = caso['event'][-1]['time:timestamp'] - caso['event'][0]['time:timestamp']
        caso['delta_time'] = delta.days
        # intervalo medio de tempo entre eventos
        caso['avg_event_interval'] = len(caso['event']) / float(delta.days)

        casos_year.append(caso)

# ordenando os casos por data de inicio
casos_year.sort(key=lambda x: x['start_time'])
print "Total de casos apos remocao: ", len(casos_year)

Total de casos apos remocao:  14507


#### Coletando apenas os eventos

In [6]:
eventos = []
for caso in casos_year:
    eventos += caso['event']
print "Total de eventos: ", len(eventos)

Total de eventos:  852610


#### Coletando apenas os atributos dos casos e salvando em arquivo

In [7]:
import csv

# removendo os eventos (preparando os casos)
for caso in casos_year:
    caso.pop('event')

# verificando caso com o maior numero de atributos
count_attr = [len(j.keys()) for j in casos_year]
# indice do caso
idx_max = count_attr.index(max(count_attr))

# convertendo dict em arquivo csv (keys ~> colunas)
keys = casos_year[idx_max].keys()
print "%s features found" % len(keys)
with open('casos_attr_%s.csv' % year, 'wb') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(casos_year)

62 features found
